<a href="https://colab.research.google.com/github/gnuevo/Intro-to-GANs/blob/master/GaussianGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GANs -- Machine Learning Tokyo Workwhop

__Mustafa Yagmur, Gregorio Nuevo Castro, Dimitris Katsios,  Suzana Ilic, Alisher Abdulkhaev
 -- June 30, 2018__

_This tutorial is based on [this](https://medium.com/@devnag/generative-adversarial-networks-gans-in-50-lines-of-code-pytorch-e81b79659e3f) code by [Dev Nag](https://medium.com/@devnag)._

This notebook is available here [https://colab.research.google.com/drive/1KcY0MwxmadHKNhSRCgM6ZuUH8k0kM41-]()

# Introduction

Welcome to this notebook. This is part of the material provided for the GAN Workwhop by _Machine Learning Tokyo_. In it we will explore some  ot the tools we will use during the workwhop:

+ **Colab**, this Jupyter-notebook like environment (https://colab.research.google.com/)
+ **PyTorch**, it is a deep learning framework with a flexible and frendly interface (https://pytorch.org/). We'll explain some aspects of PyTorch, but this is not an introduction/tutorial to the framework, so if you need extra information you can have a look at this tutorial https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html. Moreover, PyTorch is a pretty common framework nowadays, so if you have any question, Google it!

Moreover, you'll do your first implementation of a very simple GAN. Yes! For real, a GAN! It may look frightening but believe me, a simple GAN is not **that** difficult to code. _Disclaimer, if you think that any GAN is easy to code you're very much wrong XD _

## Structure of this document

This document is structured in several sections:

+ Install PyTorch and import dependencies
+ Explanation of the exercise
+ Brief introduction to GANs
+ Playing with noise and target distributions
+ Explanation of the code
    + D and G
    + Training loop
    + Hyperparameters

## Install dependencies

Before starting to code anything we have to install PyTorch in our VM in colab. It's not difficult, we can use `pip` inside colab, so just run the following cell and it'll install the framework. It'll take some minutes, be patient ;)

In [0]:
!pip install torch torchvision

## Import dependencies

Now that we have PyTorch installed we can import **all** the dependencies that we're using throughout this tutorial

+ `torch` and `torch.nn` are imports from the framework.
+ `numpy` the almighty scientific computing package for python.
+ `matplotlib` for visualisations.
+ `google.colab.widgets` contains some utilities for colab itself, we're going to use it, for example, to structure the output in grids and stuff. It's not necessary that you understand this.

In [0]:
# necesary imports

import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from google.colab import widgets
import matplotlib.gridspec as gridspec
from matplotlib import animation, rc
from IPython.display import HTML
from PIL import Image

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

So, are they imported? I hope so! 

# Proposed exercise

In this case we're going to program a very simple GAN. We want to approximate a [Gaussian function](https://en.wikipedia.org/wiki/Gaussian_function) in one dimension. In the image below you can see some examples of Gaussians for different _means_ and _standard deviations_

![Gaussian function](https://upload.wikimedia.org/wikipedia/commons/thumb/7/74/Normal_Distribution_PDF.svg/1080px-Normal_Distribution_PDF.svg.png)

Normally GANs use a Gaussian function as the `z` noise as input to the generator. In this case we're going to input Uniform noise. Why? Well, if the generator has to approximate a Gaussian function and we give it a Gaussian function as input to start with it seems a pretty easy problem, don't you think? By giving it an Uniform function, it has to find the way to apply some non-linear transformation to give it a Gaussian shape.


## Starting with the GAN

If you've read the other material linked with this code, you should be already somehow familiar with the concept of GANs and how they work (more or less). As a summary, we need 2 networks that we call **Discriminator** and **Generator**. The **Generator** tries to generate samples like those in the dataset. For example, if the dataset only contains images of dogs and cats, then the generator will try to learn how to create dogs and cats. The **Discrimintar** tries to disginguish whether a certain sample is real or fake. Real means it came from the dataset, fake it means it was generated by the **Generator**. The **Discriminator** tries to detect the images coming from the **Generator**, whereas the **Generator** tries to fool the **Discriminator** and generate images it thinks are real images of cats and dogs.

As we know, a GAN is defined by a series of elements:

+ A **target distribution**, this is the probability distribution that we want to match. In our toy example this will be a Gaussian function. In a real scenario the propability distribution will be determined by the samples in our dataset (for example images of cats and dogs).
+ The **noise sampler** (also called _prior_ in some papers), this is the input to our generator. The generator learns to convert noise samples into samples coming from the target distribution. In a real world example, this noise is usually sampled from a Gaussian distribution. In our case, however, we'll be sampling from an Uniform distribution. This distributions can have as many dimensions as we want. THIS IS AN IMPORTANT THING KEEP IT IN MIND.
+ The **Discriminator** is a network that learns to distinguish real data (coming from our target distribution) from fake data (coming from the generator)
+ The **Generator** is a network that learns to convert uncorrelated noise into samples resembling the probability distribution of our data or our target distribution.

It may seem confusing what **Discriminator** or **Generator** really are. They do cool stuff that no other networks can do, is there anything special about them? It isn't! They are just regular neural networks. Concretely, the **Discriminator** is a binary classifier: is this sample real (1) or fake (0)? And the **Generator** is a network that takes a noise sample and outputs a sample with the shape of our target distribution. Both can be different types of networks: feed forward, convolutional, etc. In our case we use simple feed forward networks.
They work because the example is simple, for mor complex problems we should find a better suited architecture.

### Target distribution and input noise

The first thing that we have to understand when coding any neural network is our data. So let's have a look at our target distribution and our noise distribution:

+ **Target distribution** Gaussian function. You'll get to choose the _mean_ and _std_ ;)
+ **noise distribution** Uniform samples between 0 and 1

It's important to get familiar with what's the _shape_ of the data we're using.

In [0]:
### Uniform sampler ###
# Our noise sampler (Z) is a function that returns a vector of a desired length 
# with uniform samples between 0 and 1
# we can use m and n to ask for a matrix of Uniform samples with the desired shape
# therefore m is the dimension of the noise
def uniform_sampler():
  return lambda m, n: torch.rand(m, n)

# we set the sampler to a variable so we can change it later if we want
noise_sampler = uniform_sampler()
# this is a 10x5 matrix with Uniform samples
uniform_matrix = noise_sampler(10, 5)



### Gaussian sampler ###
# Our Gaussian sampler is a function that returns a vector of Gaussian samples
# with a specific mean and std
# We can use n to ask for a vector of shape 1xn
def gaussian_sampler(mu, sigma):
  return lambda n: torch.Tensor(np.random.normal(mu, sigma, (1, n)))

# again, set it to a variable so we can change it later
mu, sigma = 2, 1
data_sampler = gaussian_sampler(mu, sigma)
# this is a 1x5 vector with Gaussian samples
gaussian_vector = data_sampler(5)

Let's have a look at the data. For that we can use histograms

What you'll see next is a colab form. It allows you to change the values of the variables with a friendly user interface.

+ **num_samples**, number of samples drawn from the distributions. The more samples you draw the more they'll look like what they are
+ **gaussian_mean** and **gaussian_std**, parameters of the Gaussian
+ **num_bins**, number of bars of the histogram
+ **colors...**, well... give it a try ;)

Please, experiment with different values of the number of samples, the parameters of the Gaussian, and the number of bins. Make sure you understand what's going on. You can't change the characteristics of the Uniform distribution because that our input noise, it's going to be fixed.

In [0]:
#@title Define parameters
num_samples = 8581 #@param {type:"slider", min:1, max:10000, step:10}
gaussian_mean = -0.2 #@param {type:"slider", min:-3, max:3, step:0.2}
gaussian_std = 2.1 #@param {type:"slider", min:0.1, max:3, step:0.1}

num_bins = 91 #@param {type:"slider", min: 1, max:100, step:1}
my_favourite_color_is = 'red' #@param ["green", "blue", "red", "yellow", "black", "cyan", "magenta"] {allow-input: false}
but_I_also_like = 'blue' #@param ["green", "blue", "red", "yellow", "black", "cyan", "magenta"] {allow-input: false}

# this creates a grid to make 2 plots side by side
grid = widgets.Grid(1,2)

# generate samples with the chosen characteristics
noise = noise_sampler(1, num_samples)
noise.view(num_samples).numpy()
data_sampler = gaussian_sampler(gaussian_mean, gaussian_std)
data = data_sampler(num_samples)

# plot our data
with grid.output_to(0, 0):
  n, bins, patches = plt.hist(noise, num_bins, normed=1, facecolor=my_favourite_color_is,
                                alpha=0.75)
  ax = plt.gca()
  ax.set_xlim(-0.25,1.25)
  ax.set_ylim(0, 1.5)
  plt.title("Noise distribution (Uniform between 0 and 1)")
  
with grid.output_to(0, 1):
  n, bins, patches = plt.hist(data, num_bins, normed=1, facecolor=but_I_also_like,
                                alpha=0.75)
  ax = plt.gca()
  ax.set_xlim(-10,10)
  ax.set_ylim(0, 0.7)
  plt.title("Data distribution (Gaussian with mean={} and std={})".format(gaussian_mean, gaussian_std))


I hope now you understand a bit better what we want to do. We can use a GAN to approximate a different Gaussian every time.

## Define the networks

Now it's time to define the **Discriminator** and the **Generator**. As we said, they are simple feed forward networks

### Discriminator

The discriminator has to distinguish whether a sample is real (1) or fake (0). Does it sound familiar? It recalls what a binary classifier does, right? Indeed, we model the **Discriminator** as a binary classifier.

Tip. Imagine for a moment that you're the **Discriminator**, and you have to distinguish whether a single sample comes from the real distribution or from the fake (generated) distribution. It seems like a difficult task, right? Do the following experiment. Use the above form and set `num_bins=10`. Now set `num_samples=1`. Execute the code and check what you see in the Gaussian plot on the right. Re run the code several times to see several samples. Do you see a Gaussian function? Clearly not. Maybe if you run enough times you'll understand that it is indeed a Gaussian function. But that's because you (as a human) have memory. Our **Discriminator** has not memory, so it can't remember past samples. Now increase `num_samples=100`. Re run the code several times again. Do you see a Gaussian now? Yes, you do, rigth? Maybe not a very good one (the more `num_samples` and `num_bins` the better is the Gaussian) but a Gaussian funciton indeed.

A similar thing happens to the **Discriminator**, it's very difficult for it to distinguish if the samples are coming from the real or fake distributions if we give it samples from both distributions once at a time. To overcome this problem, what is usually done is to group a set of samples that come from the distributions and give them to the **Discriminator**. For example, we take 100 samples from the real data and we give them to the **Discriminator**, therefore it can have a look at the overall structure of the data. After that, we take another 100 samples from the fake distribution and we repeat the process. This is an usefull trick, we're goint to use it in our code: instead of feeding samples individually to the **Discriminator**, we're going to create groups of 100 samples in which all of them are real or all of them are fake.

The Discriminator will look at a batch of samples and tell us if they look like comming from the target data distribution or if they look like fake data.

Let's build a Discriminator with the following characteristics:

+ It accepts an input of size `batch_size` (this is that _group_ of samples we talked about)
+ It has 3 fully connected layers
+ The activations are `elu`, except for the last layer, as it is a classifier, the last activation should be `sigmoid`

In [0]:
def discriminator(input_size=100, hidden_size=50, output_size=1):
  return nn.Sequential(
      nn.Linear(input_size, hidden_size), nn.ELU(),   # this is the 1st layer
      nn.Linear(hidden_size, hidden_size), nn.ELU(),  # this is the 2nd layer
      nn.Linear(hidden_size, 1), nn.Sigmoid()         # this is the 3rd layer
  )

### Generator

On the other side, the Generator just takes a vector of random noise as input and outputs a single sample. We can code a Generator similar to our Discriminator. So, our Generator

+ Accepts an input of size `batch_size` (this is that _group_ of samples we talked about). Each of those samples comes from the noise Uniform distribution and can have n dimensions. So, for example, we can create a set of `batch_size` noise samples, each of one has dimensionality 1, 2, 3, 10, etc.
+ It has 3 fully connected layers
+ The activations are `elu` in the middle layers
+ The output layer has no activation. Why? Well, because we want our output to be non restricted. We have to model a Gaussian function, that is defined from `-infinite` to `+infinite`. If we apply an activation function we're restricting that output. For example, `sigmoid` only outputs `[0,1]`, `tanh` outputs `[-1,1]`, `ReLU` outptus `[0, +infinite]`, and so on.

In [0]:
def generator(input_size=1, hidden_size=50, output_size=1):
  return nn.Sequential(
      nn.Linear(input_size, hidden_size), nn.ELU(),    # this is the 1st layer
      nn.Linear(hidden_size, hidden_size), nn.ELU(),   # this is the 2nd layer
      nn.Linear(hidden_size, output_size)              # this is the 3rd layer
  )

## Training

So we have all the elements defined. Now here it comes the complicated part. The training loop is probably one of the more complicated things to code and certanly what makes a GAN to be a GAN. After instantiating the necessary variables we're going to

+ Train the Discriminator to distinguish samples from the real and fake distributions. Using one group or batch from each of those distributions at a time.
+ Train the Generator to generate samples to get realistic images every time

### Explanation of the code

When you see the code for the `main()` function you'll probably get crazy, throw your computer through the window, run away and never look back to that GAN stuff that you tried to learn once. Because of that, I'll try to explain here what the code is doing. Pay attentioin and that will help you to understand it much faster.

The code is structured in several parts. On the first level, there's the (a) initialisation of variables and parameters, and (b) the trainin loop.

#### Initialisation

We instantiate

+ Our network: Discriminator (D) and Generator (G)
+ The optimisers: they are in charge of uptading the weights of G and D, and therefore getting better over time
+ The loss function: in this case we use binary crossentropy (because it is a classification task: real/fake)
+ Our samplers, both from the target and noise distribution

#### Training loop

##### Train D

We first train D with some real and fake samples

With real samples
+ Zero previous gradients with `zero_grad()`
+ Get some **real samples** (`batch_size` of them)
+ Compute the score the discriminator gives to them
+ Compute the loss. In this case all the samples are **real**. So we want the discriminator to output 1 (this is our target), so we use a vector containing a single 1 with `torch.ones_like()`.
+ Compute the loss gradients with `backward()`
+ Optimise the parameters

With fake samples we have a very similar process
+ Zero previous gradients with `zero_grad()`
+ Get some **fake samples** using the generator and the noise sampler for that (`batch_size` of them)
+ Compute the score the discriminator gives to them
+ Compute the loss. In this case all the samples are **fake**. So we want the discriminator to output 0 (this is our target), so we use a vector containing a single  with `torch.zeros_like()`.
+ Compute the loss gradients with `backward()`
+ Optimise the parameters

##### Train G

We generate some random samples using G

+ We create `batch_size` samples with G like we've been done before
+ We score those samples using D
+ Now we have to compute a loss like above. But what is our target? 0 or 1? They are generated samples, therefore the target should we 0. But we would like the Discriminator to tell us that those are real samples (1), that would mean that the Generator is doing great. Indeed, that's what we're going to do. We're going to say that our target is 1, we pretend these are real samples. And actualise the weights of G accordingly. This means, our loss and gradients are answering the question

```
How do the weights of G should change so the generated samples will be considered a bit closer to a real sample (1) by D?.
```

+ Compute the loss gradients with `backward()`
+ Optimise the parameters

Notice that D and G are never optimised at a time. If we train D with some samples, we never train G on the same samples, and vice versa.

### Hyperparameters

Like before, we're going to set some hyperparameters in a colab form so we can change them easily

+ **iterations**, for how long are we training the GAN?
+ **mu** and **sigma**, the parameters of the Gaussian
+ **input_size**, dimensionality  of the input noise.

**input_size** is a very important parameter, we'll see later. Please, for now leave these values as default and continue. You'll have time to hack them later ;)

Also notice there are other parameters that are not included in the form. So you have to change them in the code. Leave them as they are for now as well.

```python
# print some results every 200 steps
print_step = 200

# these are some hyperparameters to for the network architecture
hidden_size = 50
output_size = 1
learning_rate = 2e-4
batch_size = 100
test_num = 10000 # number of samples used for testing
```

In [0]:
#@title Define hyperparameters
iterations = 19000 #@param {type:"slider", min:1000, max:50000, step:2000}
mu = 1.5 #@param {type:"slider", min:-5, max:5, step:0.1}
sigma = 1 #@param {type:"slider", min:0.1, max:3, step:0.1}
input_size = 1 #@param {type:"slider", min:1, max:10, step:1}

# print some results every 200 steps
print_step = 200

# these are some hyperparameters to for the network architecture
hidden_size = 50
output_size = 1

learning_rate = 2e-4
batch_size = 100
test_num = 10000 # number of samples used for testing

### Utility functions

Here we're going to define some utility functions. They are useful for visualisation purposes but they are not vital for the GAN to run. So you can give it a look, but don't worry if you don't fully understand them. The important is the `train()` function below.

In [0]:
# utility function to plot the histograms of training
def make_histogram(data_real, data_fake, i, n_bins=100):
    fig = plt.figure(figsize=(8, 8))
    axes = plt.subplot(111)
    n, bins, patches = plt.hist(data_real, n_bins, normed=1, facecolor='green',
                                alpha=0.75, label="Real data")
    n, bins, patches = plt.hist(data_fake, bins, normed=1, facecolor='blue',
                                alpha=0.75, label="Generated data")
    axes.set_xlim(-5, 8)
    axes.set_ylim(0, 1)
    plt.title("Probability distribution of real and fake data")
    plt.legend(["Real data", "Generated data"])
    plt.tight_layout()

# utility function to plot the evolution of the errors during training
def plot_errors(real_loss_buffer, fake_loss_buffer, g_loss_buffer):
    fig = plt.figure(0, figsize=(10, 10))
    plt.subplot(311)
    plt.plot(real_loss_buffer)
    plt.title("D real loss")

    plt.subplot(312)
    plt.plot(fake_loss_buffer)
    plt.title("D fake loss")

    plt.subplot(313)
    plt.plot(g_loss_buffer)
    plt.title("G fake loss")

### Actual training function

In [0]:

train_gaussian = None
train_generation = None
train_errors = None
count = 0
def train():
    # ==========================================================================
    # INITIALISATION
    # ==========================================================================
    
    # this variable is for display purposes, you can ignore it!
    grid = widgets.Grid(2,2)
    global train_gaussian, train_generation, train_errors
    train_generation = []
    train_errors = []
  
    # instantiate Generator and Discriminator
    G = generator(input_size, hidden_size, output_size)
    D = discriminator(batch_size, hidden_size, output_size)
    # set the optimisers
    g_optimizer = torch.optim.Adam(G.parameters(), lr=learning_rate)
    d_optimizer = torch.optim.Adam(D.parameters(), lr=learning_rate)

    # loss function: Binary CrossEntropy
    loss_function = nn.BCELoss()

    # get our data and noise samplers
    data_sampler = gaussian_sampler(mu, sigma)
    test_data = data_sampler(test_num)
    train_gaussian = test_data

    # this list will keep track of the errors during training
    real_loss_buffer = []
    fake_loss_buffer = []
    g_loss_buffer = []
    
    global count
    count = 0
    # ==========================================================================
    # TRAINING LOOP
    # ==========================================================================
    for i in range(iterations):
        # ========================================
        # TRAIN D
        # ========================================
        """TRAINING D
            Remember, first we train D on real samples
            + delete previous gradients
            + get real samples
            + score them with D
            + compute the loss with target=1
            + compute gradients and optimise
        """
        # zero the gradients
        D.zero_grad()

        # take real samples
        real_data = data_sampler(batch_size)
        # pass the real samples through the discriminator
        real_score = D(real_data)
        # derive loss on real samples
        real_loss = loss_function(real_score, torch.ones_like(real_score))
        # save loss for visualisation purposes
        real_loss_buffer.append(real_loss)

        # compute gradients and optimize
        real_loss.backward()
        d_optimizer.step()

        """TRAINING D
            Now we train D with fake samples
            + delete previous gradients
            + get fake samples using G
            + score them with D
            + compute the loss with target=0
            + compute gradients and optimise
        """
        D.zero_grad()

        # feed fake images
        noise_data = noise_sampler(batch_size, input_size)
        fake_data = G(noise_data)

        #compute scores(logits)
        fake_score = D(fake_data.t())

        #derive loss on fake samples
        fake_loss = loss_function(fake_score, torch.zeros_like(fake_score))
        # save loss for visualisation purposes
        fake_loss_buffer.append(fake_loss)

        # compute gradients and optimize
        fake_loss.backward()
        d_optimizer.step()

        # ========================================
        # TRAIN G
        # ========================================
        """TRAINING G
            + delete previous gradients
            + get fake samples using G
            + score them with D
            + compute the loss with target=1
            + compute gradients and optimise
        """
        G.zero_grad()

        # prepare fake images through Generator
        noise_data = noise_sampler(batch_size, input_size)
        fake_data = G(noise_data)
        
        # compute scores(logits)
        g_score = D(fake_data.t())
        
        #derive loss on fake samples
        g_loss = loss_function(g_score, torch.ones_like(g_score))
        # save loss for visualisation purposes
        g_loss_buffer.append(g_loss)
        
        # compute gradients and optimize
        g_loss.backward()
        g_optimizer.step()

        
        # ========================================
        # VISUALISE TRAINING
        # (you don't have to understand this)
        # ========================================
        # print training statistics every print_step
        if (i + 1) % print_step == 0:
            # print current errors
            with grid.output_to(1,0):
                if (i + 1) % (15 * print_step) == 0:
                    grid.clear_cell()
                print("{:>6} -> D_real_loss {:.6f}  D_fake_loss {:.6f}  "
                      "G_fake_loss {:.6f}".format(
                    i+1,
                    real_loss_buffer[-1],
                    fake_loss_buffer[-1],
                    g_loss_buffer[-1]
                ))
                
            # plot distributions
            fake_data = torch.Tensor([])
            for _ in range(test_num // batch_size):
                noise_data = noise_sampler(batch_size, input_size)
                fake_data = torch.cat((fake_data, G(noise_data).t()), 1)
            fake_data = fake_data[:, :test_num]
            train_generation.append(fake_data)
            with grid.output_to(0, 0):
                grid.clear_cell()
                make_histogram(test_data.data[0,:].numpy(), fake_data.data[
                    0,:].numpy(), i)
                plt.savefig("figure_{}.png".format(count))
                count += 1
                
            # plot training errors
            with grid.output_to(0, 1):
                grid.clear_cell()
                plot_errors(real_loss_buffer, fake_loss_buffer, g_loss_buffer)
                
    train_errors = [real_loss_buffer, fake_loss_buffer, g_loss_buffer]

## Run it

Now that everything is defined, it's time to run it.

_Note: it's not going to work very well at the first run. Don't let it affect you, we'll improve that later_

In [0]:
# fingers crossed
train()

### Analysis of the results

So wow! What's that?! Are you kidding me!? All this effort (_effort in a figurative way, you could have gotten to here just by pressing Shift+Enter_ :|)  to get such a shitty distribution (blue)?

Yes! Nobody said this was easy. GANs are not easy stuff to train. Certainly we can make some improvements.

But before going further, did you understand what's going on?

+ Plot on the left shows the probability distribution of both real and fake data during training. So you can see how the Generator is training. Do you see that is usually always changing? And sometimes looks like it's getting stable and suddenly it changes a lot? Well, that's one of the issues with GANs, **stability** is a problem.

+ Plot on the right shows the evolution of the errors, from top to bottom
  + **D real loss**, the error of D when classifying real samples, the lower it is the better **real** samples are classified as real
  + **D fake loss**, the error of D when classifying fake samples, the lower it is the better **fake** samples are classified as real
  + **G fake loss**, the error G samples, the lower it is the better is G fooling D.

  But what happend here? What a crazy plot, right?! Indeed, it's difficult to understand the meaning of the error plots of GANs. If you're familiar with neural networks you've probably seen many accuracy or error plots. In which the error starts very high and decreases over time, or the accuracy starts very low and increases over time. Well, here is not like that. Why, because the error of D depends on G (if G gets better then the error will tend to raise); but at the same time the error of G depends on D (if D gets better, it is going to catch G easier and therefore output a greater error). But as both D and G improve over time, there is no baseline we can use to compute an objective accuracy. That's why normally the performance of the GANs has to be assessed manually. Definitive conclusions can't be extracted from the error measures.

## Let's fix this

Ok, I said it wasn't going to be good at the beginning. But I also said we could fix it. And we indeed can. The key is in the hyperparameters. Do you remember that I wrote `THIS IS AN IMPORTANT THING KEEP IT IN MIND` somewhere above. Well, if I did it's probably because `THAT IS AN IMPORTANT THING KEEP IT IN MIND`.

We'll discuss this further during the workshop, but the noise Z creates a latent space of variables. The GAN just learns to give meaning or structure to that latent space. Therefore, its dimension is important. Higher dimensions, more expressivity that the GAN can have. It'll learn to somehow correlate the dimensions in Z to give it some meaning when generating a sample.

So what I want you to know is to try and change the dimensionality of Z. The variable that does this is `input_size`. Start by doing `input_size=2` and re-run the training code. 

Don't worry, I've copyed the _form_ for you again here. So you just have to select the new values for the variables and run the cell. It'll make the GAN to train with the new configuration.

In [0]:
#@title Define hyperparameters
iterations = 45000 #@param {type:"slider", min:1000, max:50000, step:2000}
mu = -0.3 #@param {type:"slider", min:-5, max:5, step:0.1}
sigma = 0.6 #@param {type:"slider", min:0.1, max:3, step:0.1}
input_size = 3 #@param {type:"slider", min:1, max:10, step:1}

# print some results every 200 steps
print_step = 200

# these are some hyperparameters to for the network architecture
hidden_size = 50
output_size = 1

learning_rate = 2e-4
batch_size = 100
test_num = 10000 # number of samples used for testing


# better luck this time
train()

### Better results

Wow! Things improved, didn't they? Mmmm... I wonder what would happen if we use even more input dimensions. If only there was a way of figuring it out... I guess we'll never now! :P

## Further things to try

By now you should have  a general idea of how things work. But how do all the rest of the hyperparameters affect the training. I don't know. I don't even know to which extent is possible to predic its effect. So let's try it, shall we?

Of course, we also encourage you to change `mu` and `sigma` of the Gaussian to fit different target distributions.

Appart from the variables in the form, there are other ones you can also change, for example.

```python
hidden_size = 50
learning_rate = 2e-4
batch_size = 100
```

Have fun, see all of you in the workshop :)

## Do a funny video of the training

Now we're finally going to do a video with all the training information. During the execution of the program wi silently stored all the plots in disk as images. Their names are `figure_0.png`, `figure_1.png` and so on. Check that the files are actually there with `ls`

In [0]:
!ls
!ls | wc -l
print("Number of images saved", count)

If the previous command fails you may need to restart the runtime. Do it under the `Runtime` menu on the menu bar on the top.

**Note: In case the runtime is restarted, we must import all the modules again. It is also needed to the variable `count` to its right value (the number of images saved during training). Luckly, the following script should fix this. Just click the checkbox and run.**

In [0]:
#@title Did you have to restart? (check and run if you did)
restart_checkbox = False #@param {type:"boolean"}

if restart_checkbox:
  # import again
  print("Importing modules")
  import torch
  from torch import nn
  import numpy as np
  import matplotlib.pyplot as plt
  from google.colab import widgets
  import matplotlib.gridspec as gridspec
  from matplotlib import animation, rc
  from IPython.display import HTML
  from PIL import Image
  
  # read files
  print("Reading the number of images")
  from os import listdir
  count = len(list(filter(lambda name: "figure_" in name, listdir())))
  print("Number of images saved", count)

The following cell defines the logic that takes those images and creates the animation.

In [0]:
class AnimObject(object):
    def __init__(self, images):
        print(len(images))
        self.fig, self.ax = plt.subplots()
        self.ax.set_title("")
        self.fig.set_size_inches((8, 8))
        self.plot = plt.imshow(images[0])
        plt.tight_layout()
        self.images = images
        
    def init(self):
        self.plot.set_data(self.images[0])
        self.ax.grid(False)
        return (self.plot,)
      
    def animate(self, i):
        self.plot.set_data(self.images[i])
        self.ax.grid(False)
        self.ax.set_xticks([])
        self.ax.set_yticks([])
        self.ax.set_title("index {}".format(i))
        return (self.plot,)

def get_figures(template, indices):
    import os.path
    images = []
    for index in indices:
        if os.path.isfile(template.format(index)):
            images.append(Image.open(template.format(index)))
    return images
  
images = get_figures("figure_{}.png", range(count))
animobject = AnimObject(images)
anim = animation.FuncAnimation(
              animobject.fig,
              animobject.animate,
              frames=len(animobject.images),
              interval=150,
              blit=True)


### Visualise the animation

Finally we can just convert the animation to html so we can watch it in the browser.

In [0]:
HTML(anim.to_jshtml())